# Handle the CIFAR-10 dataset

https://www.dlology.com/blog/quick-notes-on-how-to-choose-optimizer-in-keras/

In [0]:
%matplotlib inline
import pickle
import numpy as np
from os import listdir
from os.path import isfile, join
import os
from sklearn.preprocessing import StandardScaler

## Create a function that will extract the data from the dataset
You can find the dataset here: https://www.cs.utoronto.ca/~kriz/cifar-10-python.tar.gz

When you download the dataset you will get a bunch of pickled files <br>
Inside `cifar-10-batches-py`folder you will find <br>
1. data_batch_x files <br>
2. test_batch 

The data_batch_x files are the batched training data splited in x parts (in our case in 5 parts of 10000 samples each). The test_batch file contains 10000 samples for testing.

### The stracture
All files has the same structure: <br>
1. data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
2. labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

There is also another file called batches.meta where it describes the classes

In [0]:
# Function to unpickle the dataset
def unpickle_all_data(directory):
    
    # Initialize the variables
    train = dict()
    test = dict()
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    
    # Iterate through all files that we want, train and test
    # Train is separated into batches
    for filename in listdir(directory):
        if isfile(join(directory, filename)):
            
            # The train data
            if 'data_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Opent the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo)

                if 'data' not in train:
                    train['data'] = data[b'data']
                    train['labels'] = np.array(data[b'labels'])
                else:
                    train['data'] = np.concatenate((train['data'], data[b'data']))
                    train['labels'] = np.concatenate((train['labels'], data[b'labels']))
            # The test data
            elif 'test_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Open the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo)
                
                test['data'] = data[b'data']
                test['labels'] = data[b'labels']
    
    # Manipulate the data to the propper format
    for image in train['data']:
        train_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    train_y = [label for label in train['labels']]
    
    for image in test['data']:
        test_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    test_y = [label for label in test['labels']]
    
    # Transform the data to np array format
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    
    return (train_x, train_y), (test_x, test_y)

## Using the function

Returns four variables
1. x_train := A 3D numpy array with the images in a format (32, 32, 3) a 32x32 image with 3 dimmesions for colors RGB
2. y_train := A 1D numpy with the numbered labels for each sample stom x_train
3. x_test := same as x_train
4. y_test := same as y_train

In [3]:
#/home/pragya/Desktop/IIITBangalore/Semester3/AVR/Tensorflow_AVR/sagemaker-deep-learning-master/cifar-10-keras-mxnet/sagemaker
from keras.datasets import cifar10 
#(x_train, y_train), (x_test, y_test) = unpickle_all_data(os.getcwd() + '/cifar-10-batches-py/')
(x_train, y_train), (x_test, y_test)=cifar10.load_data()

Using TensorFlow backend.


170508288/170498071 [==============================] - 60s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


## Preprocess the data
1. The data must be in a float32 represenation
2. The data should be normalized, here I chose [0,1] normalization
3. The labels are one-hot encoded

In [0]:
from keras.utils import np_utils

# Transofrm them to a float32 type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the input 
x_train /= 255
x_test /= 255

# One-hot Encoding
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


## Building the model
1. The input data for the model is each image sample 32x32x3
2. Convolve the image into 3x3 squares with padding same (https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t) and Pooling with 2x2. Add a dropout of 0.2 to avoid early overfitting
3. Same situation for the next two layers for 64 input size, and 128 input size
4. Finally, add a Dense(fully connected) layer for the output

You can find more information about the model here: https://appliedmachinelearning.wordpress.com/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/

You can try different layouts to improve the accuracy

In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(num_classes, activation='softmax'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

## Augment the data
From one sample, extract multiple versions

For example, rotate the image, shift, flip etc

In [0]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

## Compiling the model

In [0]:
# Compile the model
batch_size = 64

#opt_rms = keras.optimizers.rmsprop(lr=0.001, decay=1e-6) for momentum instead of adam use oppt_rms
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

## Training the algorithm
I would suggest more than 100 epochs

In [10]:
import time
start=time.time()
epochs = 30
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
end=time.time()

Epoch 1/30
781/781 [==============================] - 39s 50ms/step - loss: 1.6591 - acc: 0.4141 - val_loss: 1.2649 - val_acc: 0.5483
Epoch 2/30
781/781 [==============================] - 37s 47ms/step - loss: 1.2767 - acc: 0.5395 - val_loss: 1.0373 - val_acc: 0.6326
Epoch 3/30
781/781 [==============================] - 36s 46ms/step - loss: 1.1574 - acc: 0.5868 - val_loss: 1.2556 - val_acc: 0.5691
Epoch 4/30
781/781 [==============================] - 36s 46ms/step - loss: 1.0886 - acc: 0.6155 - val_loss: 0.9664 - val_acc: 0.6549
Epoch 5/30
781/781 [==============================] - 35s 45ms/step - loss: 1.0403 - acc: 0.6326 - val_loss: 0.9906 - val_acc: 0.6512
Epoch 6/30
781/781 [==============================] - 36s 46ms/step - loss: 1.0022 - acc: 0.6467 - val_loss: 0.9021 - val_acc: 0.6798
Epoch 7/30
781/781 [==============================] - 36s 46ms/step - loss: 0.9793 - acc: 0.6548 - val_loss: 0.8991 - val_acc: 0.6761
Epoch 8/30
781/781 [==============================] - 36s 46ms

In [11]:
end-start

1087.4245779514313